# Group Work AS 2025: ESG Investment Analysis and ML Portfolio Optimization

This notebook provides the data analysis and machine learning implementation for the Group Work AS 2025 assignment. 

**For complete assignment details, deliverables, and requirements, please refer to the [Group Assignment PDF](https://github.com/umatter/EDFB/blob/main/_groupwork_dev/prev_groupwork/groupwork_AS_2025/Group_assignment_AS_2025.pdf).**

## Dataset Details:
- **Source**: ESG & Financial Performance Dataset by Shriyash Jagtap (Kaggle)
- **License**: Creative Commons Attribution 4.0 International (CC BY 4.0)
- **Original URL**: https://www.kaggle.com/datasets/shriyashjagtap/esg-and-financial-performance-dataset
- **Size**: 11,000 observations across 1,000 companies (2015-2025)
- **Variables**: Company info, financial metrics, ESG scores, environmental impact data
- **Format**: CSV file loaded directly from GitHub repository

## Instructions:
1. Make a copy of this notebook (File → Save a copy in Drive)
2. Run each cell sequentially
3. The dataset is automatically loaded from the GitHub repository
4. Use the analysis results for your written report and presentation
5. Submit the completed notebook along with your other deliverables

**Important**: This notebook uses real ESG data under CC BY 4.0 license. Please maintain proper attribution in your work.

# 1. Data Loading and Initial Setup

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.decomposition import PCA

# Portfolio optimization
from scipy.optimize import minimize

# Settings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")
%matplotlib inline

In [ ]:
# Load the ESG dataset from GitHub repository
# Dataset: ESG & Financial Performance Dataset by Shriyash Jagtap
# Source: https://www.kaggle.com/datasets/shriyashjagtap/esg-and-financial-performance-dataset
# License: CC BY 4.0 (https://creativecommons.org/licenses/by/4.0/)

print("Loading ESG & Financial Performance Dataset...")
print("Dataset Attribution:")
print("- Author: Shriyash Jagtap")
print("- License: Creative Commons Attribution 4.0 International (CC BY 4.0)")
print("- Source: https://www.kaggle.com/datasets/shriyashjagtap/esg-and-financial-performance-dataset")
print("-" * 80)

# Load data directly from GitHub repository
github_data_url = "https://raw.githubusercontent.com/umatter/EDFB/main/data/company_esg_financial_dataset.csv"

try:
    # Load the dataset
    esg_data = pd.read_csv(github_data_url)
    
    # Rename columns to match the expected format in the rest of the notebook
    column_mapping = {
        'CompanyID': 'Company_ID',
        'CompanyName': 'Company_Name', 
        'GrowthRate': 'Growth_Rate',
        'ESG_Overall': 'ESG_Score',
        'ESG_Environmental': 'Environmental_Score',
        'ESG_Social': 'Social_Score',
        'ESG_Governance': 'Governance_Score',
        'ProfitMargin': 'Profit_Margin',
        'MarketCap': 'Market_Cap',
        'CarbonEmissions': 'Carbon_Emissions',
        'WaterUsage': 'Water_Usage',
        'EnergyConsumption': 'Energy_Consumption'
    }
    
    esg_data = esg_data.rename(columns=column_mapping)
    
    # Handle missing values in Growth_Rate column (first year for each company)
    esg_data['Growth_Rate'] = esg_data['Growth_Rate'].fillna(0)
    
    print(f"✓ Dataset loaded successfully!")
    print(f"✓ {len(esg_data):,} observations")
    print(f"✓ {esg_data['Company_Name'].nunique():,} companies")  
    print(f"✓ Time period: {esg_data['Year'].min()}-{esg_data['Year'].max()}")
    print(f"✓ Industries: {esg_data['Industry'].nunique()}")
    print(f"✓ Regions: {esg_data['Region'].nunique()}")
    
except Exception as e:
    print(f"❌ Error loading dataset: {e}")
    print("\nFallback: Creating sample dataset for demonstration...")
    
    # Fallback to sample data if GitHub load fails
    sample_data = {
        'Company_ID': [1, 1, 2, 2],
        'Company_Name': ['Company_1', 'Company_1', 'Company_2', 'Company_2'],
        'Industry': ['Technology', 'Technology', 'Healthcare', 'Healthcare'],
        'Region': ['North America', 'North America', 'Europe', 'Europe'],
        'Year': [2023, 2024, 2023, 2024],
        'Revenue': [1000, 1100, 800, 850],
        'Profit_Margin': [15.5, 16.2, 12.8, 13.1],
        'Market_Cap': [5000, 5500, 4000, 4200],
        'Growth_Rate': [10.0, 8.5, 6.2, 7.1],
        'ESG_Score': [75.2, 76.8, 68.5, 69.9],
        'Environmental_Score': [78.1, 79.2, 65.3, 67.1],
        'Social_Score': [72.8, 74.5, 71.2, 72.8],
        'Governance_Score': [74.9, 76.7, 69.0, 69.4],
        'Carbon_Emissions': [25000, 24000, 28000, 27500],
        'Water_Usage': [15000, 14500, 18000, 17800],
        'Energy_Consumption': [45000, 44000, 50000, 49500]
    }
    esg_data = pd.DataFrame(sample_data)
    print("Sample dataset created for testing purposes.")

print("\n" + "="*80)
esg_data.head()

# 2. Data Exploration and Quality Assessment

In [ ]:
# Check dataset dimensions and basic info
print("Dataset Shape:", esg_data.shape)
print("\nData Types:")
print(esg_data.dtypes)
print("\nBasic Information:")
esg_data.info()

In [ ]:
# Check for missing values
missing_values = esg_data.isnull().sum()
print("Missing Values per Column:")
print(missing_values[missing_values > 0] if missing_values.sum() > 0 else "No missing values found")

# Summary statistics
print("\n" + "="*50)
print("SUMMARY STATISTICS")
print("="*50)
esg_data.describe()

In [ ]:
# Distribution analysis by industry and region
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Industry distribution
industry_counts = esg_data['Industry'].value_counts()
axes[0, 0].pie(industry_counts.values, labels=industry_counts.index, autopct='%1.1f%%')
axes[0, 0].set_title('Distribution by Industry')

# Region distribution
region_counts = esg_data['Region'].value_counts()
axes[0, 1].pie(region_counts.values, labels=region_counts.index, autopct='%1.1f%%')
axes[0, 1].set_title('Distribution by Region')

# ESG Score distribution
axes[1, 0].hist(esg_data['ESG_Score'], bins=30, alpha=0.7, color='green')
axes[1, 0].set_title('Distribution of ESG Scores')
axes[1, 0].set_xlabel('ESG Score')
axes[1, 0].set_ylabel('Frequency')

# Year distribution
year_counts = esg_data['Year'].value_counts().sort_index()
axes[1, 1].bar(year_counts.index, year_counts.values)
axes[1, 1].set_title('Distribution by Year')
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Number of Companies')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# ESG Score components analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# ESG components distribution
esg_components = ['Environmental_Score', 'Social_Score', 'Governance_Score']
colors = ['green', 'blue', 'orange']

for i, (component, color) in enumerate(zip(esg_components, colors)):
    if i < 3:
        row, col = i // 2, i % 2
        axes[row, col].hist(esg_data[component], bins=30, alpha=0.7, color=color, label=component)
        axes[row, col].set_title(f'Distribution of {component.replace("_", " ")}')
        axes[row, col].set_xlabel('Score')
        axes[row, col].set_ylabel('Frequency')

# ESG trend over time
esg_trend = esg_data.groupby('Year')['ESG_Score'].mean()
axes[1, 1].plot(esg_trend.index, esg_trend.values, marker='o', linewidth=2, markersize=8)
axes[1, 1].set_title('Average ESG Score Trend Over Time')
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Average ESG Score')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 3. Outlier Detection and Treatment

In [ ]:
# Identify numerical columns for outlier analysis
numerical_cols = ['Revenue', 'Profit_Margin', 'Market_Cap', 'Growth_Rate', 
                  'ESG_Score', 'Environmental_Score', 'Social_Score', 'Governance_Score',
                  'Carbon_Emissions', 'Water_Usage', 'Energy_Consumption']

# Box plot visualization for outlier detection
fig, axes = plt.subplots(3, 4, figsize=(20, 15))
axes = axes.ravel()

for i, col in enumerate(numerical_cols):
    if i < len(axes):
        # Use log scale for financial and environmental metrics for better visualization
        if col in ['Revenue', 'Market_Cap', 'Carbon_Emissions', 'Water_Usage', 'Energy_Consumption']:
            data_to_plot = np.log1p(esg_data[col])  # log(1+x) to handle zeros
            axes[i].set_ylabel(f'Log({col})')
        else:
            data_to_plot = esg_data[col]
            axes[i].set_ylabel(col)
        
        axes[i].boxplot(data_to_plot, patch_artist=True)
        axes[i].set_title(f'Box Plot: {col}')
        axes[i].grid(True, alpha=0.3)

# Remove empty subplots
for j in range(len(numerical_cols), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
# Outlier detection using IQR method
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (df[column] < lower_bound) | (df[column] > upper_bound)

# Count outliers for each numerical column
outlier_counts = {}
for col in numerical_cols:
    outliers = detect_outliers_iqr(esg_data, col)
    outlier_counts[col] = outliers.sum()

print("Outlier Counts by Column (using IQR method):")
print("-" * 45)
for col, count in outlier_counts.items():
    percentage = (count / len(esg_data)) * 100
    print(f"{col:20s}: {count:4d} ({percentage:.1f}%)")

total_outliers = sum(outlier_counts.values())
print(f"\nTotal outlier instances: {total_outliers}")
print(f"Percentage of dataset with outliers: {(total_outliers / (len(esg_data) * len(numerical_cols))) * 100:.1f}%")

In [ ]:
# Create a cleaned dataset by capping outliers at 95th and 5th percentiles
esg_data_clean = esg_data.copy()

outlier_treatment_log = {}

for col in numerical_cols:
    original_outliers = detect_outliers_iqr(esg_data, col).sum()
    
    # Cap outliers at 5th and 95th percentiles
    lower_cap = esg_data[col].quantile(0.05)
    upper_cap = esg_data[col].quantile(0.95)
    
    esg_data_clean[col] = esg_data_clean[col].clip(lower=lower_cap, upper=upper_cap)
    
    new_outliers = detect_outliers_iqr(esg_data_clean, col).sum()
    outlier_treatment_log[col] = {
        'original_outliers': original_outliers,
        'remaining_outliers': new_outliers,
        'reduction': original_outliers - new_outliers
    }

print("Outlier Treatment Results:")
print("-" * 60)
print(f"{'Column':<20} {'Original':<10} {'Remaining':<10} {'Reduced':<10}")
print("-" * 60)
for col, stats in outlier_treatment_log.items():
    print(f"{col:<20} {stats['original_outliers']:<10} {stats['remaining_outliers']:<10} {stats['reduction']:<10}")

print(f"\nDataset shape after outlier treatment: {esg_data_clean.shape}")

# 4. Correlation Analysis

In [ ]:
# Calculate correlation matrix for key variables
correlation_vars = ['Revenue', 'Profit_Margin', 'Market_Cap', 'Growth_Rate', 
                   'ESG_Score', 'Environmental_Score', 'Social_Score', 'Governance_Score']

correlation_matrix = esg_data_clean[correlation_vars].corr()

# Create correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Mask upper triangle
sns.heatmap(correlation_matrix, annot=True, cmap='RdYlGn', center=0,
            square=True, fmt='.3f', cbar_kws={'label': 'Correlation Coefficient'},
            mask=mask)
plt.title('Correlation Matrix: Financial Performance vs ESG Scores', fontsize=16)
plt.tight_layout()
plt.show()

print("Key Correlations with ESG Score:")
print("-" * 35)
esg_correlations = correlation_matrix['ESG_Score'].drop('ESG_Score').sort_values(key=abs, ascending=False)
for var, corr in esg_correlations.items():
    print(f"{var:<20}: {corr:6.3f}")

In [ ]:
# ESG Score vs Financial Performance Scatter Plots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

financial_metrics = ['Revenue', 'Profit_Margin', 'Market_Cap', 'Growth_Rate']
colors = ['blue', 'green', 'red', 'purple']

for i, (metric, color) in enumerate(zip(financial_metrics, colors)):
    row, col = i // 2, i % 2
    
    # Use log scale for Revenue and Market_Cap for better visualization
    if metric in ['Revenue', 'Market_Cap']:
        y_data = np.log1p(esg_data_clean[metric])
        axes[row, col].set_ylabel(f'Log({metric})')
    else:
        y_data = esg_data_clean[metric]
        axes[row, col].set_ylabel(metric)
    
    axes[row, col].scatter(esg_data_clean['ESG_Score'], y_data, alpha=0.5, color=color)
    
    # Add trend line
    z = np.polyfit(esg_data_clean['ESG_Score'], y_data, 1)
    p = np.poly1d(z)
    axes[row, col].plot(esg_data_clean['ESG_Score'], p(esg_data_clean['ESG_Score']), "r--", alpha=0.8)
    
    # Calculate and display correlation
    corr = esg_data_clean['ESG_Score'].corr(y_data)
    axes[row, col].set_title(f'ESG Score vs {metric}\n(Correlation: {corr:.3f})')
    axes[row, col].set_xlabel('ESG Score')
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Industry-wise ESG and Financial Performance Analysis
industry_analysis = esg_data_clean.groupby('Industry').agg({
    'ESG_Score': ['mean', 'std'],
    'Revenue': 'mean',
    'Profit_Margin': 'mean',
    'Growth_Rate': 'mean'
}).round(2)

# Flatten column names
industry_analysis.columns = ['_'.join(col).strip() for col in industry_analysis.columns]
industry_analysis = industry_analysis.sort_values('ESG_Score_mean', ascending=False)

print("Industry-wise ESG and Financial Performance:")
print("=" * 80)
print(industry_analysis)

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# ESG Score by Industry
industry_esg = esg_data_clean.groupby('Industry')['ESG_Score'].mean().sort_values(ascending=True)
axes[0, 0].barh(industry_esg.index, industry_esg.values, color='green', alpha=0.7)
axes[0, 0].set_title('Average ESG Score by Industry')
axes[0, 0].set_xlabel('Average ESG Score')

# Revenue by Industry
industry_revenue = esg_data_clean.groupby('Industry')['Revenue'].mean().sort_values(ascending=True)
axes[0, 1].barh(industry_revenue.index, industry_revenue.values / 1e6, color='blue', alpha=0.7)
axes[0, 1].set_title('Average Revenue by Industry')
axes[0, 1].set_xlabel('Average Revenue (Millions)')

# Profit Margin by Industry
industry_margin = esg_data_clean.groupby('Industry')['Profit_Margin'].mean().sort_values(ascending=True)
axes[1, 0].barh(industry_margin.index, industry_margin.values, color='orange', alpha=0.7)
axes[1, 0].set_title('Average Profit Margin by Industry')
axes[1, 0].set_xlabel('Average Profit Margin (%)')

# Growth Rate by Industry
industry_growth = esg_data_clean.groupby('Industry')['Growth_Rate'].mean().sort_values(ascending=True)
axes[1, 1].barh(industry_growth.index, industry_growth.values, color='red', alpha=0.7)
axes[1, 1].set_title('Average Growth Rate by Industry')
axes[1, 1].set_xlabel('Average Growth Rate (%)')

plt.tight_layout()
plt.show()

# 5. Machine Learning Models for Return Prediction

In [ ]:
# Prepare data for machine learning
# We'll predict Growth_Rate (as a proxy for returns) using ESG scores and other features

# Encode categorical variables
le_industry = LabelEncoder()
le_region = LabelEncoder()

ml_data = esg_data_clean.copy()
ml_data['Industry_encoded'] = le_industry.fit_transform(ml_data['Industry'])
ml_data['Region_encoded'] = le_region.fit_transform(ml_data['Region'])

# Define features (X) and target (y)
feature_cols = ['ESG_Score', 'Environmental_Score', 'Social_Score', 'Governance_Score',
                'Revenue', 'Profit_Margin', 'Market_Cap', 'Year', 
                'Industry_encoded', 'Region_encoded']

X = ml_data[feature_cols]
y = ml_data['Growth_Rate']

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"\nFeatures: {feature_cols}")
print(f"Target: Growth_Rate (as proxy for returns)")

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")
print(f"\nTarget variable statistics:")
print(f"Mean: {y.mean():.2f}%")
print(f"Std: {y.std():.2f}%")
print(f"Min: {y.min():.2f}%")
print(f"Max: {y.max():.2f}%")

In [ ]:
# Train multiple machine learning models
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

model_results = {}

print("Model Training and Evaluation Results:")
print("=" * 60)

for name, model in models.items():
    # Train model
    if name == 'Random Forest':
        model.fit(X_train, y_train)  # Tree-based models don't need scaling
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
    else:
        model.fit(X_train_scaled, y_train)
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)
    
    # Calculate metrics
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    
    model_results[name] = {
        'model': model,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'predictions_train': y_pred_train,
        'predictions_test': y_pred_test
    }
    
    print(f"\n{name}:")
    print(f"  Training R²: {train_r2:.4f}")
    print(f"  Testing R²:  {test_r2:.4f}")
    print(f"  Training RMSE: {train_rmse:.4f}")
    print(f"  Testing RMSE:  {test_rmse:.4f}")
    print(f"  CV R² Score: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")

# Select best model based on test R²
best_model_name = max(model_results.keys(), key=lambda x: model_results[x]['test_r2'])
best_model = model_results[best_model_name]

print(f"\n" + "="*60)
print(f"Best Model: {best_model_name} (Test R² = {best_model['test_r2']:.4f})")
print("="*60)

In [ ]:
# Feature importance analysis for Random Forest
rf_model = model_results['Random Forest']['model']
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Feature Importance (Random Forest):")
print("-" * 40)
for _, row in feature_importance.iterrows():
    print(f"{row['feature']:<20}: {row['importance']:.4f}")

# Visualize feature importance
plt.figure(figsize=(12, 8))
plt.barh(feature_importance['feature'][::-1], feature_importance['importance'][::-1])
plt.title('Feature Importance for Growth Rate Prediction (Random Forest)', fontsize=16)
plt.xlabel('Importance Score')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Model performance visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Actual vs Predicted plots for both models
for i, (name, results) in enumerate(model_results.items()):
    # Training set
    axes[i, 0].scatter(y_train, results['predictions_train'], alpha=0.5)
    axes[i, 0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--')
    axes[i, 0].set_xlabel('Actual Growth Rate (%)')
    axes[i, 0].set_ylabel('Predicted Growth Rate (%)')
    axes[i, 0].set_title(f'{name} - Training Set\n(R² = {results["train_r2"]:.4f})')
    axes[i, 0].grid(True, alpha=0.3)
    
    # Testing set
    axes[i, 1].scatter(y_test, results['predictions_test'], alpha=0.5, color='orange')
    axes[i, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    axes[i, 1].set_xlabel('Actual Growth Rate (%)')
    axes[i, 1].set_ylabel('Predicted Growth Rate (%)')
    axes[i, 1].set_title(f'{name} - Testing Set\n(R² = {results["test_r2"]:.4f})')
    axes[i, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 6. Portfolio Optimization with ESG Constraints

In [ ]:
# Portfolio optimization using Modern Portfolio Theory with ESG constraints
# We'll create portfolios based on different ESG criteria

# Get latest year data for portfolio construction
latest_year = esg_data_clean['Year'].max()
portfolio_data = esg_data_clean[esg_data_clean['Year'] == latest_year].copy()

# Select top companies by different criteria for portfolio construction
n_assets = 50  # Number of assets in portfolio

# Portfolio 1: Traditional - highest returns (Growth Rate)
traditional_portfolio = portfolio_data.nlargest(n_assets, 'Growth_Rate')

# Portfolio 2: ESG-focused - highest ESG scores
esg_portfolio = portfolio_data.nlargest(n_assets, 'ESG_Score')

# Portfolio 3: Balanced - combination of returns and ESG
portfolio_data['Combined_Score'] = (0.6 * portfolio_data['Growth_Rate'].rank(pct=True) + 
                                   0.4 * portfolio_data['ESG_Score'].rank(pct=True))
balanced_portfolio = portfolio_data.nlargest(n_assets, 'Combined_Score')

portfolios = {
    'Traditional': traditional_portfolio,
    'ESG-Focused': esg_portfolio,
    'Balanced': balanced_portfolio
}

print(f"Portfolio Analysis ({latest_year} data, {n_assets} assets each):")
print("=" * 70)

portfolio_stats = {}
for name, data in portfolios.items():
    stats = {
        'avg_return': data['Growth_Rate'].mean(),
        'return_std': data['Growth_Rate'].std(),
        'avg_esg': data['ESG_Score'].mean(),
        'esg_std': data['ESG_Score'].std(),
        'avg_revenue': data['Revenue'].mean(),
        'avg_margin': data['Profit_Margin'].mean(),
        'sharpe_ratio': data['Growth_Rate'].mean() / data['Growth_Rate'].std() if data['Growth_Rate'].std() > 0 else 0
    }
    portfolio_stats[name] = stats
    
    print(f"\n{name} Portfolio:")
    print(f"  Average Return: {stats['avg_return']:.2f}% (σ = {stats['return_std']:.2f}%)")
    print(f"  Average ESG Score: {stats['avg_esg']:.1f} (σ = {stats['esg_std']:.1f})")
    print(f"  Sharpe Ratio: {stats['sharpe_ratio']:.3f}")
    print(f"  Average Revenue: ${stats['avg_revenue']/1e6:.1f}M")
    print(f"  Average Profit Margin: {stats['avg_margin']:.1f}%")

In [ ]:
# Portfolio comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Risk-Return scatter plot
for name, stats in portfolio_stats.items():
    axes[0, 0].scatter(stats['return_std'], stats['avg_return'], 
                      s=200, alpha=0.7, label=name)
    axes[0, 0].annotate(name, (stats['return_std'], stats['avg_return']), 
                       xytext=(5, 5), textcoords='offset points')

axes[0, 0].set_xlabel('Risk (Return Std Dev %)')
axes[0, 0].set_ylabel('Expected Return (%)')
axes[0, 0].set_title('Risk-Return Profile by Portfolio Type')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# ESG vs Return comparison
portfolio_names = list(portfolio_stats.keys())
esg_scores = [portfolio_stats[p]['avg_esg'] for p in portfolio_names]
returns = [portfolio_stats[p]['avg_return'] for p in portfolio_names]

axes[0, 1].bar(portfolio_names, esg_scores, alpha=0.7, color='green')
axes[0, 1].set_ylabel('Average ESG Score', color='green')
axes[0, 1].set_title('ESG Scores by Portfolio Type')
axes[0, 1].tick_params(axis='y', labelcolor='green')

# Create second y-axis for returns
ax2 = axes[0, 1].twinx()
ax2.plot(portfolio_names, returns, color='red', marker='o', linewidth=2, markersize=8)
ax2.set_ylabel('Average Return (%)', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Industry diversification comparison
for i, (name, data) in enumerate(portfolios.items()):
    industry_dist = data['Industry'].value_counts()
    if i == 0:
        axes[1, 0].pie(industry_dist.values, labels=industry_dist.index, 
                      autopct='%1.1f%%', startangle=90)
        axes[1, 0].set_title(f'{name} Portfolio - Industry Distribution')
    elif i == 1:
        axes[1, 1].pie(industry_dist.values, labels=industry_dist.index, 
                      autopct='%1.1f%%', startangle=90)
        axes[1, 1].set_title(f'{name} Portfolio - Industry Distribution')

plt.tight_layout()
plt.show()

# Additional industry distribution for Balanced portfolio
plt.figure(figsize=(10, 8))
industry_dist = portfolios['Balanced']['Industry'].value_counts()
plt.pie(industry_dist.values, labels=industry_dist.index, autopct='%1.1f%%', startangle=90)
plt.title('Balanced Portfolio - Industry Distribution')
plt.axis('equal')
plt.show()

In [ ]:
# Efficient Frontier calculation (simplified version)
# Calculate expected returns and covariance matrix for efficient frontier

def calculate_portfolio_metrics(weights, returns, cov_matrix):
    portfolio_return = np.sum(weights * returns)
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_return, portfolio_std

# Use the traditional portfolio companies for efficient frontier
ef_data = traditional_portfolio[['Company_Name', 'Growth_Rate']].set_index('Company_Name')

# Simulate historical returns (in reality, you would use actual historical data)
n_periods = 12  # 12 months of data
np.random.seed(42)

# Create synthetic return series for each company
returns_data = pd.DataFrame(index=range(n_periods), columns=ef_data.index)
for company in ef_data.index:
    expected_return = ef_data.loc[company, 'Growth_Rate'] / 12  # Monthly return
    volatility = abs(expected_return) * 0.5  # Assume volatility is 50% of return
    returns_data[company] = np.random.normal(expected_return, volatility, n_periods)

# Calculate expected returns and covariance matrix
expected_returns = returns_data.mean()
cov_matrix = returns_data.cov()

# Generate random portfolios for efficient frontier
n_portfolios = 10000
results = np.zeros((3, n_portfolios))
np.random.seed(42)

for i in range(n_portfolios):
    # Generate random weights
    weights = np.random.random(len(expected_returns))
    weights /= np.sum(weights)  # Normalize to sum to 1
    
    # Calculate portfolio metrics
    port_return, port_std = calculate_portfolio_metrics(weights, expected_returns, cov_matrix)
    
    # Store results
    results[0, i] = port_return * 12  # Annualize
    results[1, i] = port_std * np.sqrt(12)  # Annualize
    results[2, i] = results[0, i] / results[1, i]  # Sharpe ratio

# Create DataFrame for results
ef_results = pd.DataFrame({
    'Return': results[0],
    'Volatility': results[1], 
    'Sharpe': results[2]
})

# Plot efficient frontier
plt.figure(figsize=(12, 8))
scatter = plt.scatter(ef_results['Volatility'], ef_results['Return'], 
                     c=ef_results['Sharpe'], cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Sharpe Ratio')

# Highlight max Sharpe ratio portfolio
max_sharpe_idx = ef_results['Sharpe'].idxmax()
plt.scatter(ef_results.loc[max_sharpe_idx, 'Volatility'], 
           ef_results.loc[max_sharpe_idx, 'Return'], 
           marker='*', s=500, color='red', label='Max Sharpe Ratio')

plt.xlabel('Volatility (%)')
plt.ylabel('Expected Return (%)')
plt.title('Efficient Frontier - Traditional Portfolio Assets')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Optimal Portfolio (Max Sharpe Ratio):")
print(f"Expected Return: {ef_results.loc[max_sharpe_idx, 'Return']:.2f}%")
print(f"Volatility: {ef_results.loc[max_sharpe_idx, 'Volatility']:.2f}%")
print(f"Sharpe Ratio: {ef_results.loc[max_sharpe_idx, 'Sharpe']:.3f}")

# 7. ESG Impact Analysis and Investment Implications

In [ ]:
# ESG Impact Analysis
# Analyze how ESG scores have evolved over time and their impact on financial performance

# ESG score evolution over time
esg_evolution = esg_data_clean.groupby('Year').agg({
    'ESG_Score': 'mean',
    'Environmental_Score': 'mean',
    'Social_Score': 'mean', 
    'Governance_Score': 'mean',
    'Growth_Rate': 'mean',
    'Profit_Margin': 'mean'
})

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# ESG component evolution
axes[0, 0].plot(esg_evolution.index, esg_evolution['ESG_Score'], marker='o', linewidth=2, label='Overall ESG')
axes[0, 0].plot(esg_evolution.index, esg_evolution['Environmental_Score'], marker='s', linewidth=2, label='Environmental')
axes[0, 0].plot(esg_evolution.index, esg_evolution['Social_Score'], marker='^', linewidth=2, label='Social')
axes[0, 0].plot(esg_evolution.index, esg_evolution['Governance_Score'], marker='d', linewidth=2, label='Governance')
axes[0, 0].set_title('ESG Score Evolution Over Time')
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Average Score')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# ESG vs Financial Performance over time
ax1 = axes[0, 1]
ax2 = ax1.twinx()

line1 = ax1.plot(esg_evolution.index, esg_evolution['ESG_Score'], 'g-o', linewidth=2, label='ESG Score')
line2 = ax2.plot(esg_evolution.index, esg_evolution['Growth_Rate'], 'b-s', linewidth=2, label='Growth Rate')

ax1.set_xlabel('Year')
ax1.set_ylabel('ESG Score', color='g')
ax2.set_ylabel('Growth Rate (%)', color='b')
ax1.set_title('ESG Score vs Financial Performance Over Time')

# Combine legends
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper left')
ax1.grid(True, alpha=0.3)

# ESG quartile analysis
latest_data = esg_data_clean[esg_data_clean['Year'] == latest_year]
latest_data['ESG_Quartile'] = pd.qcut(latest_data['ESG_Score'], 4, labels=['Q1 (Low)', 'Q2', 'Q3', 'Q4 (High)'])

quartile_performance = latest_data.groupby('ESG_Quartile').agg({
    'Growth_Rate': 'mean',
    'Profit_Margin': 'mean',
    'Revenue': 'mean'
})

quartile_performance['Growth_Rate'].plot(kind='bar', ax=axes[1, 0], color='skyblue', alpha=0.7)
axes[1, 0].set_title('Average Growth Rate by ESG Quartile')
axes[1, 0].set_ylabel('Growth Rate (%)')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3)

quartile_performance['Profit_Margin'].plot(kind='bar', ax=axes[1, 1], color='lightcoral', alpha=0.7)
axes[1, 1].set_title('Average Profit Margin by ESG Quartile')
axes[1, 1].set_ylabel('Profit Margin (%)')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("ESG Quartile Performance Analysis:")
print("=" * 50)
print(quartile_performance.round(2))

In [ ]:
# Environmental impact correlation analysis
environmental_impact = esg_data_clean[['ESG_Score', 'Environmental_Score', 
                                      'Carbon_Emissions', 'Water_Usage', 'Energy_Consumption']]

# Calculate correlations
env_corr = environmental_impact.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(env_corr, annot=True, cmap='RdYlGn', center=0, square=True, fmt='.3f')
plt.title('Environmental Impact vs ESG Scores Correlation Matrix')
plt.tight_layout()
plt.show()

print("Key Environmental Correlations:")
print("-" * 40)
env_esg_corr = env_corr['Environmental_Score'].drop('Environmental_Score').sort_values(key=abs, ascending=False)
for var, corr in env_esg_corr.items():
    print(f"{var:<20}: {corr:6.3f}")

# 8. Now it's your turn!

## For Your Written Report and Presentation: See the Group_assignment_AS_2025_content.pdf for detailed instructions.
